In [1]:
# import utils and modules
import StochasticGradientAscentUtils as sga
import derivatives as der
import activationFunctions as activate
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import norm
import os,glob
from six.moves import cPickle as pickle


In [2]:
def oneHotEncoding(labels):
    maxval = np.max(labels)
    return np.eye(maxval+1)[labels]

# Read and store our data from CIFAR-10 


In [3]:
# store our file names for better usage
def make_filePaths_to_arr():
    print("Trying to find all files in cifar-10-batches-py directory!")
    files = glob.glob('*/*')
    CIFAR = [f for f in files if "cifar" in f and "." not in f]   #store only the batch_data
    return CIFAR
        

In [4]:
# split array into test_paths and train_paths
files = make_filePaths_to_arr()
files

Trying to find all files in cifar-10-batches-py directory!


['cifar-10-batches-py\\data_batch_1',
 'cifar-10-batches-py\\data_batch_2',
 'cifar-10-batches-py\\data_batch_3',
 'cifar-10-batches-py\\data_batch_4',
 'cifar-10-batches-py\\data_batch_5',
 'cifar-10-batches-py\\test_batch']

In [5]:
train_file_paths = [path for path in files[:5]]
test_file_paths = files[-1]

In [6]:
def load_image_data():
    batches = []
    for file in train_file_paths:
        with open(file, 'rb') as f:
            print("Loading: " + file)
            data = pickle.load(f, encoding='latin1')
            X = data['data']
            Y = oneHotEncoding(data['labels'])
            X = X.reshape(10000,3072)
            Y = np.array(Y)
            batch = [X,Y]
            batches.append(batch)
    with open(test_file_paths, 'rb') as f:
            print("Loading: " + test_file_paths)
            data = pickle.load(f, encoding='latin1')
            X_test = data['data']
            Y_test = oneHotEncoding(data['labels'])
            X_test = X_test.reshape(10000,3072)
            Y_test = np.array(Y_test)
            
    return batches, X_test, Y_test        

In [7]:
def concatData():
    batches, x_test, y_test = load_image_data()
    X_train, Y_train = [], []
    for batch in batches:
        batchX = batch[0]
        batchY = batch[1]
        X_train.extend(batchX)
        Y_train.extend(batchY)
        
    return np.array(X_train), np.array(Y_train), x_test, y_test
        

## Normalize train, test

In [8]:
X_train, Y_train, X_test, Y_test = concatData()
X_train = X_train.astype(float)/255
X_test = X_test.astype(float)/255

Loading: cifar-10-batches-py\data_batch_1
Loading: cifar-10-batches-py\data_batch_2
Loading: cifar-10-batches-py\data_batch_3
Loading: cifar-10-batches-py\data_batch_4
Loading: cifar-10-batches-py\data_batch_5
Loading: cifar-10-batches-py\test_batch


# Gradcheck for random hyperParameters

In [9]:
%%time
gradW1 , num_w1, gradW2, num_w2 = sga.gradcheck(X_train, Y_train, 0.1, 200, 100, 10, "h3")

Wall time: 46min 28s


In [10]:
print( "Gradient of W1 is : ", np.max(np.abs(gradW1 - num_w1)) )

Gradient of W1 is :  1.3028934375824974e-08


In [11]:
print( "Gradient of W2 is : ", np.max(np.abs(gradW2 - num_w2)) )

Gradient of W2 is :  1.0419002771655528e-08


# Testing hyperParameters for CIFAR-10 data

In [12]:
def testParameters(X, T, x_test, y_test):
    
    X_Test = np.copy(x_test)
    
    add_one = np.ones((X_Test.shape[0],1))
    X_Test = np.concatenate((add_one,X_Test), axis=1)
    
    lamdas = [0.1, 0.5]
    learning_rates = [0.01, 0.001]
    EPOCHS = [10, 20]
    Ms = [100, 200, 300]
    activation_h = ["h1", "h2", "h3"]
    batch_size = [200]
    
    results = {"batch_size" : [],
              "HiddenLayers M" : [],
              "activation_h" : [],
              "epochs" : [],
              "learning_rate" : [],
              "lamda" : [],
               "error" : [],
              "accuracy" : []
              }
    
    scores = []
    for batchSize in batch_size:
        for M in Ms:
            for h in activation_h:
                for epoch in EPOCHS:
                    print("Current M: ", M, " | Current h: ", h, " | Current epoch: ", epoch)
                    for l_rate in learning_rates:
                        for l in lamdas:
                            w1, w2 ,costs = sga.mini_batches_SGA(X, T,activation_h = h, hiddenLayers = M, classesK = 10, learning_rate = l_rate, lamda = l, epochs = epoch, batch_size = batchSize)
                            pred, _, _ = sga.NeuralNetwork(X_Test, w1, w2, hiddenLayerSize = M, outLayerSize = 10, activation_h = h)
                            acc = np.mean( np.argmax(pred,1) == np.argmax(y_test ,1) )
                            scores.append(acc) # save score we got
                            results['batch_size'].append(batchSize)
                            results['HiddenLayers M'].append(M)
                            results['activation_h'].append(h)
                            results['epochs'].append(epoch)
                            results['learning_rate'].append(l_rate)
                            results['lamda'].append(l)
                            results['error'].append(1-acc)
                            results['accuracy'].append(acc)
    return scores, results

In [13]:
%%time
scores, data = testParameters(X_train, Y_train, X_test, Y_test)

# save results
df = pd.DataFrame(data, columns= ['batch_size', 'HiddenLayers M', 'activation_h', 'epochs', 'learning_rate', 'lamda', 'error', 'accuracy'])
df.to_csv("CIFAR_RESULTS_TEST_PARAMETERS.csv")

Current M:  100  | Current h:  h1  | Current epoch:  10
Current M:  100  | Current h:  h1  | Current epoch:  20
Current M:  100  | Current h:  h2  | Current epoch:  10
Current M:  100  | Current h:  h2  | Current epoch:  20
Current M:  100  | Current h:  h3  | Current epoch:  10
Current M:  100  | Current h:  h3  | Current epoch:  20
Current M:  200  | Current h:  h1  | Current epoch:  10
Current M:  200  | Current h:  h1  | Current epoch:  20
Current M:  200  | Current h:  h2  | Current epoch:  10
Current M:  200  | Current h:  h2  | Current epoch:  20
Current M:  200  | Current h:  h3  | Current epoch:  10
Current M:  200  | Current h:  h3  | Current epoch:  20
Current M:  300  | Current h:  h1  | Current epoch:  10
Current M:  300  | Current h:  h1  | Current epoch:  20
Current M:  300  | Current h:  h2  | Current epoch:  10
Current M:  300  | Current h:  h2  | Current epoch:  20
Current M:  300  | Current h:  h3  | Current epoch:  10
Current M:  300  | Current h:  h3  | Current epo